In [1]:
%load_ext autoreload
%autoreload 2

import time
import cv2
import numpy as np
import tensorflow as tf
import imutils
import pandas as pd
from src.handDetector import HandDetector

objc[5384]: Class CaptureDelegate is implemented in both /opt/homebrew/Caskroom/miniforge/base/envs/env_tf/lib/python3.8/site-packages/cv2/cv2.abi3.so (0x152ece480) and /opt/homebrew/Caskroom/miniforge/base/envs/env_tf/lib/python3.8/site-packages/mediapipe/.dylibs/libopencv_videoio.3.4.16.dylib (0x1639f4860). One of the two will be used. Which one is undefined.
objc[5384]: Class CVWindow is implemented in both /opt/homebrew/Caskroom/miniforge/base/envs/env_tf/lib/python3.8/site-packages/cv2/cv2.abi3.so (0x152ece4d0) and /opt/homebrew/Caskroom/miniforge/base/envs/env_tf/lib/python3.8/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x16288ca68). One of the two will be used. Which one is undefined.
objc[5384]: Class CVView is implemented in both /opt/homebrew/Caskroom/miniforge/base/envs/env_tf/lib/python3.8/site-packages/cv2/cv2.abi3.so (0x152ece4f8) and /opt/homebrew/Caskroom/miniforge/base/envs/env_tf/lib/python3.8/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.1

4.6.0


In [2]:
# Get this info from the trainer's notebook
classes = {
    "ok": 0,
    "victory": 1,
    "call": 2,
    "hang_in": 3,
    "one_up": 4,
    "two_up": 5,
    "hand_closed": 6,
    "hand_open": 7,
    "machedici": 8,
    "random": 9,
}
classes = {v: k for k, v in classes.items()}
classes


{0: 'ok',
 1: 'victory',
 2: 'call',
 3: 'hang_in',
 4: 'one_up',
 5: 'two_up',
 6: 'hand_closed',
 7: 'hand_open',
 8: 'machedici',
 9: 'random'}

In [3]:
def put_text(img, text):

  font = cv2.FONT_HERSHEY_SIMPLEX
  fontScale = 0.5
  color = (255, 0, 0)
  thickness=1

  y0, dy = 15, 15
  for i, line in enumerate(text):
    y = y0 + i*dy
    cv2.putText(img, line, (5, y ), font, fontScale, color, thickness, cv2.LINE_AA)

## Prediction Logic

In [4]:
tflite_save_path = 'models/lite.hdf5'
interpreter = tf.lite.Interpreter(model_path=tflite_save_path)
interpreter.allocate_tensors()
def prediction(arr):

  arr = arr.astype(np.float32)
  # Get I / O tensor
  input_details = interpreter.get_input_details()
  output_details = interpreter.get_output_details()

  interpreter.set_tensor(input_details[0]['index'], np.array([arr]))

  interpreter.invoke()
  tflite_results = interpreter.get_tensor(output_details[0]['index'])

  sq = np.squeeze(tflite_results)
  idx = np.argmax(sq)
  pred = sq[idx]
  return idx, pred

## Stuff to do

In [5]:
from src.teams_interaction import TeamsInteractions

teams = TeamsInteractions()

def do_your_stuff(classes, idx_pred):


    if classes[idx_pred] == "call":
        teams.meet_call_click()

    if classes[idx_pred] == "ok":
        teams.meet_send_reaction_click("ok")

    if classes[idx_pred] == "hand_open":
        teams.meet_send_reaction_click("raisehand")
        

## Main Loop

In [10]:

ESC_KEY = 27
WAIT_KEY = 30
MIN_SCORE = 0.8

detector = HandDetector()


cap = cv2.VideoCapture(0)


while cap.isOpened():

    success, img = cap.read()
    if not success:
      break

    k = cv2.waitKey(WAIT_KEY)
    if k == ESC_KEY:
      break

    img = imutils.resize(img, width=800)
   

    detector.fit(img)
    img = detector.transform_draw(img)
    img = detector.transform_connect_lines(img)
    img = detector.transform_square(img)

    if detector.found_hands():
      arr = detector.get_positions_normalized()[0]
      idx_pred, pred = prediction(arr)

      if pred >= MIN_SCORE and classes[idx_pred] != "random":
        put_text(img, [str(classes[idx_pred]), str(pred)])

        do_your_stuff(classes, idx_pred)

   
    cv2.imshow("Image", img)

cv2.destroyAllWindows()
cv2.waitKey(1)
cap.release()

Not found button on screen: meet_unmute.png
Not found button on screen: meet_unmute.png
Not found button on screen: meet_unmute.png
Not found button on screen: meet_unmute.png
          Display Type: Built-In Retina LCD
          Resolution: 2560 x 1600 Retina
Not found button on screen: meet_unmute.png
          Display Type: Built-In Retina LCD
          Resolution: 2560 x 1600 Retina
Not found button on screen: meet_unmute.png
Not found button on screen: meet_unmute.png
Not found button on screen: meet_unmute.png
Not found button on screen: meet_unmute.png
Not found button on screen: meet_unmute.png
          Display Type: Built-In Retina LCD
          Resolution: 2560 x 1600 Retina
Not found button on screen: meet_mute.png


In [9]:
classes

{0: 'ok',
 1: 'victory',
 2: 'call',
 3: 'hang_in',
 4: 'one_up',
 5: 'two_up',
 6: 'hand_closed',
 7: 'hand_open',
 8: 'machedici',
 9: 'random'}